# Equity Research

An equity research analyst is a professional who evaluates public companies and provides recommendations on whether to buy, sell, or hold their stocks.

**Buy-Side Analysts** work for institutional investors such as hedge funds ...
- [ ] Please complete a bit more of a blurb here

**Sell-side Analysts** work for brokerage firms and investment banks ...
- [ ] Please complete a bit more of a blurb here

Today we are going to put ourselves in the shoes of a **Sell-Side** Equity Analyst. 

We are going to look at some of the tools we can use in Python to assist in building an equity research report for publication.
We are not so much focussed on the process of formulating the analysis such as building a DCF.
Rather we want to learn how to use the tools that can automate some of the work we may do as an analyst.

Step 1 will be to create a few handy functions

Step 2 will be to pull in some live data from an application programming interface (API)

- [ ] Please describe in successive slide fragments all the things we are going to do in the lecture

- [ ] Please describe in successive slide fragments all the things we are going to do in the lecture

# Create some handy functions
Here we are going to create some functions that we can call to do some calculations

In [ ]:
# Function to calculate the P/E ratio of a stock
def calculate_pe_ratio(price, earnings):
    return price / earnings

# Function to calculate the dividend yield of a stock
def calculate_dividend_yield(dividend, price):
    return dividend / price

# Tutor to add more functions here
# put each function in a separate cell so that each can be brought up individually in the slideshow

# Application Programming Interfaces (APIs)

- [ ] Convert my slides from last year into Jupyter Slide Show cells (you may need to create .jpg files for the diagrams and render them in the slideshow)

In [ ]:
# Use the joke of the day API and the weather API examples from last years slides to show how to use APIs in Python 
# these are contained in API_examples.ipynb

- [ ] describe the yFinance library as accessing the Yahoo finance API to get current data for a stock

In [ ]:
# pull some data on an australian stock from yfinance
# and feed this data into the functions above to calculate some ratios etc


In [ ]:
# use the read_html() function from pandas to scrape some data from a website that is relevant to the stock
# again, you can refer to an example from API_examples.ipynb


In [ ]:
# use ipywidgets to create an interactive widget that allows the user to input the stock code 
# this will then pull the data from yfinance and run the functions above to calculate the ratios

In [8]:
import ipywidgets as widgets
from IPython.display import display

# Create a text input widget
stock_code_input = widgets.Text(
    value='',
    placeholder='Stock code',
    description='Stock code:',
    disabled=False
)

# Create a button widget
submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

# Define the function to handle button click
def on_button_click(b):
    stock_code = stock_code_input.value
    print(f'Entered Stock Code: {stock_code}')

# Attach the function to the button click event
submit_button.on_click(on_button_click)

# Display the widgets
display(stock_code_input, submit_button)

Text(value='', description='Stock code:', placeholder='Stock code')

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Click me')

In [12]:
# print the user input
print(f'User entered stock code: {stock_code_input.value}')

User entered stock code: BHP
